In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 7
TITLE = "23-24 December 2020"
SUBTITLE = "8 AM 24 December 2020"
sts = utc(2020, 12, 24, 6)
ets = utc(2020, 12, 26, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [13]:
cull = ['IA-PA-5', 'LSXI4', 'Z5', 'WLBI4', 'ALEI2', 'OSKI4', 'GNDI4']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=0.1)
res = mp.postprocess(filename='201224.png')
mp.close()

    state  wfo     val        lon        lat                             geo  \
192    NE  OAX  0.0001 -96.088900  40.236100  POINT (332127.278 -521716.781)   
211    MO  EAX  0.0001 -93.608600  40.082400  POINT (543594.320 -525643.866)   
212    IA  DMX  0.0001 -94.910200  40.800600  POINT (428375.407 -453631.061)   
213    IA  DMX  0.0001 -94.462062  41.038713  POINT (464313.525 -424782.548)   
215    MO  EAX  0.0001 -95.386600  40.437300  POINT (390500.444 -496313.719)   
218    MO  EAX  0.0001 -93.583900  40.398600  POINT (543082.681 -490423.137)   
219    WI  ARX  0.0001 -90.837200  43.134300  POINT (741668.803 -165765.511)   
222    MO  EAX  0.0001 -92.770300  39.896100  POINT (616405.057 -540418.182)   
228    MO  EAX  0.0001 -94.026900  40.257500  POINT (506754.041 -508883.490)   
230    MO  DVN  0.0001 -92.166700  40.460800  POINT (662006.546 -473190.125)   
241    IA  DVN  0.0001 -91.970000  40.720000  POINT (675878.196 -442885.557)   
244    IA  DVN  0.0001 -90.591000  41.54